David Culhane
<br>
Term Project Milestone 3: Cleaning/Formatting Website Data
<br>
<br>
The next stage for this project will entail cleaning/data preparation for the HTML/website table data. The data for this cleaning/preparation will come from the UN and is found at https://en.wikipedia.org/wiki/List_of_countries_by_inflation_rate. The web page has been locally saved to make sure that future runs of the code rely on a localy saved copy due to the nature of Wikipedia. The UN data spans a time period from 2010-2023 with a list of countries and other statistics related to regions of the world.
<br>
<br>
**Loading the Data**
Loading the data will be done using BeautifulSoup and the goal is to take the UN data, in terms of annual percentage changes, is prepared for future merging with the other data sources.

In [3]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

# Loading the website's HTML data using with open and a file handler
with open('List of countries by inflation rate - Wikipedia.html', 'r', encoding='utf-8') as fp:
    soup = BeautifulSoup(fp)

With the HTML code loaded into the soup object, we want to load only the UN data regarding inflation rates. This table is the second wikitable on the page. So to isolate that data, we can search for all wikitable objects in the page and index for the desired table inside the find_all results.

In [5]:
UNtable = soup.find_all('table', {'class' : 'wikitable'})[1]
UNtable

<table class="mw-datatable wikitable sortable sticky-header-multi static-row-numbers sort-under jquery-tablesorter" style="text-align:right;">
<thead><tr>
<th class="headerSort" role="columnheader button" rowspan="3" tabindex="0" title="Sort ascending">Country/Territory/Region/Group
</th>
<th colspan="13">United Nations consumer price indices
</th></tr><tr>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">2010
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">2011
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">2012
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">2013
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">2014
</th>
<th class="headerSort" role="columnheader button" tabindex="0" title="Sort ascending">2015
</th>
<th class="headerSort" role="columnheader button" tabindex

With the desired data isolated, we will want to create a dataframe using the data. Ideally this can be done by taking a list of the headers to use as keys in a dictionary and the list of values for the headers going into lists that would serve as the values for the keys in the dictionary that will be turned into our dataframe.

In [7]:
headers = [th.getText().rstrip("\n") for th in UNtable.findAll('th')]
headers

['Country/Territory/Region/Group',
 'United Nations consumer price indices',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

Since the second header is technically a super-header, we can remove it for now by popping its location

In [9]:
headers.pop(1)
headers

['Country/Territory/Region/Group',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022']

Now we will want to get the values from the tables and store them into lists that will serve as the keys to our dictionary.

In [11]:
# Getting the country names
countries = []
for i in range(212):
    line = UNtable.tbody.findAll('tr', recursive=False)[i]
    country = line.find('a').getText()
    countries.append(country)

In [12]:
countries

['Aruba',
 'Afghanistan',
 'Angola',
 'Albania',
 'Andorra',
 'United Arab Emirates',
 'Argentina',
 'Armenia',
 'American Samoa',
 'Antigua and Barbuda',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Burundi',
 'Belgium',
 'Benin',
 'Burkina Faso',
 'Bangladesh',
 'Bulgaria',
 'Bahrain',
 'Bahamas',
 'Bosnia and Herzegovina',
 'Belarus',
 'Belize',
 'Bermuda',
 'Bolivia',
 'Brazil',
 'Barbados',
 'Brunei',
 'Bhutan',
 'Botswana',
 'Central African Republic',
 'Canada',
 'Switzerland',
 'Channel Islands',
 'Chile',
 'China',
 'Ivory Coast',
 'Cameroon',
 'DR Congo',
 'Congo',
 'Colombia',
 'Comoros',
 'Cape Verde',
 'Costa Rica',
 'Cuba',
 'Curaçao',
 'Cayman Islands',
 'Cyprus',
 'Czech Republic',
 'Germany',
 'Djibouti',
 'Dominica',
 'Denmark',
 'Dominican Republic',
 'Algeria',
 'Ecuador',
 'Egypt',
 'Eritrea',
 'Spain',
 'Estonia',
 'Ethiopia',
 'Finland',
 'Fiji',
 'France',
 'Faroe Islands',
 'Micronesia',
 'Gabon',
 'United Kingdom',
 'Georgia',
 'Ghana',
 'Guinea',
 'Gambia',
 'G

A function can be written to mirror the for block used to get the country names that instead indexes through the td tags for the data values for the one specific to the desired year. Since the 0th td tag has country information, the 1st has 2010 and the 13th has 2022. That is the span of the data.

In [14]:
import re # Importing re to substitute non-ascii characters with a hyphen since the negative signs in the table are unable to be converted
def extractor(index):
    set = []
    for i in range(212):
        line = UNtable.tbody.findAll('tr', recursive=False)[i]
        value = line.findAll('td')[index].getText()
        value = re.sub(r'[^\x00-\x7F]+','-', value)  # Replacing the non-asciis with -
        try:
            value = np.double(value)
        except:
            value = np.nan
        set.append(value)
    return set

In [15]:
# Getting the annual inflation data
total2010 = extractor(1)
total2011 = extractor(2)
total2012 = extractor(3)
total2013 = extractor(4)
total2014 = extractor(5)
total2015 = extractor(6)
total2016 = extractor(7)
total2017 = extractor(8)
total2018 = extractor(9)
total2019 = extractor(10)
total2020 = extractor(11)
total2021 = extractor(12)
total2022 = extractor(13)

In [16]:
# Making the dictionary
dict = {
    'Country' : countries,
    '2010' : total2010,
    '2011' : total2011,
    '2012' : total2012,
    '2013' : total2013,
    '2014' : total2014,
    '2015' : total2015,
    '2016' : total2016,
    '2017' : total2017,
    '2018' : total2018,
    '2019' : total2019,
    '2020' : total2020,
    '2021' : total2021,
    '2022' : total2022,
}

# Making the dataframe
UNdata_raw = pd.DataFrame(dict)
UNdata_raw

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,2.08,4.38,0.57,-2.37,0.42,0.47,-0.93,-1.03,3.63,3.94,-1.31,0.73,5.59
1,Afghanistan,NaN,12.17,-28.64,-9.90,4.67,-1.55,2.12,-10.57,0.63,2.30,5.60,5.13,13.71
2,Angola,2.15,4.70,1.43,0.19,-0.28,-0.94,-0.57,1.34,0.38,0.75,-0.47,1.78,3.04
3,Albania,3.61,3.44,2.04,1.93,1.61,1.87,1.29,1.98,2.03,1.41,1.62,2.04,6.73
4,Andorra,14.48,13.48,10.28,8.78,7.30,9.16,30.69,29.84,19.63,17.08,22.28,25.77,21.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,Kosovo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
208,Yemen,11.17,19.54,9.89,10.97,8.16,22.00,21.30,30.40,33.65,15.43,19.59,26.02,29.07
209,South Africa,4.21,4.99,5.65,5.74,6.10,4.59,6.33,5.28,4.61,4.12,3.28,4.56,6.87
210,Zambia,8.50,8.66,6.58,6.98,7.81,10.11,17.87,6.58,7.49,9.15,15.73,22.02,10.99


**Transformation One: Removing Blank Data**
<br>
<br>
Now that the data has finally been moved into a dataframe, we can start cleaning/preparing the data. Thhe first thing to do is remove blank data. For any given country in the dataframe, the most NaN's present are 3, when excluding Kosovo. Unfortunately, Kosovo has no data in the table but is included for some reason. Kosovo's data should be removed.

In [18]:
# Removing Kosovo
UNdata = UNdata_raw.drop([207])
UNdata

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,2.08,4.38,0.57,-2.37,0.42,0.47,-0.93,-1.03,3.63,3.94,-1.31,0.73,5.59
1,Afghanistan,NaN,12.17,-28.64,-9.90,4.67,-1.55,2.12,-10.57,0.63,2.30,5.60,5.13,13.71
2,Angola,2.15,4.70,1.43,0.19,-0.28,-0.94,-0.57,1.34,0.38,0.75,-0.47,1.78,3.04
3,Albania,3.61,3.44,2.04,1.93,1.61,1.87,1.29,1.98,2.03,1.41,1.62,2.04,6.73
4,Andorra,14.48,13.48,10.28,8.78,7.30,9.16,30.69,29.84,19.63,17.08,22.28,25.77,21.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,Samoa,-0.20,2.87,6.19,-0.21,-1.24,1.92,0.07,1.35,3.68,2.19,1.48,-3.01,8.75
208,Yemen,11.17,19.54,9.89,10.97,8.16,22.00,21.30,30.40,33.65,15.43,19.59,26.02,29.07
209,South Africa,4.21,4.99,5.65,5.74,6.10,4.59,6.33,5.28,4.61,4.12,3.28,4.56,6.87
210,Zambia,8.50,8.66,6.58,6.98,7.81,10.11,17.87,6.58,7.49,9.15,15.73,22.02,10.99


**Transformation 2: Outlier Removal**
<br>
<br>
For the data we have, each year should be considered independently for outlier idenfication. Then for each country, if it ends up as an outlier more than 5 times, it could be removed. To identify the outliers, a new dataframe made up of z-scores can be constructed. We can then measure the absolute values of the z-scores and if that value is greater than 3, it will be counted as an outlier.

In [20]:
#Defining a z-scoring function to act on the series for annual data
def zscorer(series):
    scores = []
    for i in series:
        zscore = (i - series.mean()) / series.std()
        scores.append(zscore)
    return scores


# Making the z-score dataframe
UNzs = pd.DataFrame()
UNzs['2010'] = zscorer(UNdata['2010'])
UNzs['2011'] = zscorer(UNdata['2011'])
UNzs['2012'] = zscorer(UNdata['2012'])
UNzs['2013'] = zscorer(UNdata['2013'])
UNzs['2014'] = zscorer(UNdata['2014'])
UNzs['2015'] = zscorer(UNdata['2015'])
UNzs['2016'] = zscorer(UNdata['2016'])
UNzs['2017'] = zscorer(UNdata['2017'])
UNzs['2018'] = zscorer(UNdata['2018'])
UNzs['2019'] = zscorer(UNdata['2019'])
UNzs['2020'] = zscorer(UNdata['2020'])
UNzs['2021'] = zscorer(UNdata['2021'])
UNzs['2022'] = zscorer(UNdata['2022'])
UNzs.index = UNdata['Country']
UNzs

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
Country,,,,,,,,,,,,,
Aruba,-0.506031,-0.273996,-0.676569,-1.029393,-0.513970,-0.338711,-0.237589,-0.242649,-0.069028,-0.069743,-0.121286,-0.139611,-0.317017
Afghanistan,NaN,1.067719,-5.149432,-2.219667,0.106916,-0.527226,-0.142100,-0.531594,-0.069694,-0.070940,-0.079846,-0.100637,-0.003109
Angola,-0.488664,-0.218881,-0.544879,-0.624732,-0.616233,-0.470298,-0.226318,-0.170867,-0.069750,-0.072071,-0.116249,-0.130310,-0.415596
Albania,-0.126444,-0.435898,-0.451471,-0.349689,-0.340122,-0.208056,-0.168086,-0.151483,-0.069383,-0.071589,-0.103715,-0.128007,-0.272946
Andorra,2.570360,1.293348,0.810303,0.733097,0.491135,0.472280,0.752363,0.692332,-0.065472,-0.060153,0.020185,0.082185,0.292629
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Samoa,-1.071690,-0.534072,0.184010,-0.687960,-0.756481,-0.203390,-0.206281,-0.170564,-0.069017,-0.071020,-0.104554,-0.172738,-0.194856
Yemen,1.749162,2.337096,0.750583,1.079272,0.616774,1.670568,0.458383,0.709293,-0.062357,-0.061357,0.004053,0.084400,0.590687
South Africa,0.022413,-0.168932,0.101321,0.252561,0.315826,0.045787,-0.010294,-0.051534,-0.068810,-0.069611,-0.093760,-0.105686,-0.267534


In [21]:
# Boolean filtering the z-scores to identify outliers
filter = UNzs.map(lambda x: abs(x) >= 3)

drops = []
for i in range(210):
    if filter.iloc[i].sum() >= 5:
        drops.append(i)
    else:
        continue

The drops list contains the indices of countries with 5 or more years that stand out as outliers. We can drop those countries by passing the list into the drop method.

In [23]:
UNzfilt = UNdata.drop(drops)
UNzfilt

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,2.08,4.38,0.57,-2.37,0.42,0.47,-0.93,-1.03,3.63,3.94,-1.31,0.73,5.59
1,Afghanistan,NaN,12.17,-28.64,-9.90,4.67,-1.55,2.12,-10.57,0.63,2.30,5.60,5.13,13.71
2,Angola,2.15,4.70,1.43,0.19,-0.28,-0.94,-0.57,1.34,0.38,0.75,-0.47,1.78,3.04
3,Albania,3.61,3.44,2.04,1.93,1.61,1.87,1.29,1.98,2.03,1.41,1.62,2.04,6.73
4,Andorra,14.48,13.48,10.28,8.78,7.30,9.16,30.69,29.84,19.63,17.08,22.28,25.77,21.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,Samoa,-0.20,2.87,6.19,-0.21,-1.24,1.92,0.07,1.35,3.68,2.19,1.48,-3.01,8.75
208,Yemen,11.17,19.54,9.89,10.97,8.16,22.00,21.30,30.40,33.65,15.43,19.59,26.02,29.07
209,South Africa,4.21,4.99,5.65,5.74,6.10,4.59,6.33,5.28,4.61,4.12,3.28,4.56,6.87
210,Zambia,8.50,8.66,6.58,6.98,7.81,10.11,17.87,6.58,7.49,9.15,15.73,22.02,10.99


**Transformation 3: Global Mean**
<br>
<br>
The data in the table represents the change in Consumer Price Index for the given year. The CPI itself is a weighred average, so each value in the dataframe represents the increase or decrease in the CPI from the previous year. Adding a mean row would be able to repesent a global average change per unit of local currency.

In [25]:
# Setting up the dataframe to concatenate with
columns = UNzfilt.columns

# Getting the global mean for each year
means = []
for column in columns:
    try:
        mean = UNzfilt[column].mean().round(2)
        means.append(mean)
    except:
        mean = 'Global Mean'
        means.append(mean)

# Making the dataframe
gbmeans = pd.DataFrame(means, columns).T
gbmeans

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Global Mean,4.0,5.88,4.76,3.74,3.44,3.37,5.27,4.87,4.48,5.14,7.28,8.58,12.51


In [26]:
# Concatenating the Dataframes
UN = pd.concat([UNzfilt, gbmeans], ignore_index=True)
UN

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Aruba,2.08,4.38,0.57,-2.37,0.42,0.47,-0.93,-1.03,3.63,3.94,-1.31,0.73,5.59
1,Afghanistan,NaN,12.17,-28.64,-9.9,4.67,-1.55,2.12,-10.57,0.63,2.3,5.6,5.13,13.71
2,Angola,2.15,4.7,1.43,0.19,-0.28,-0.94,-0.57,1.34,0.38,0.75,-0.47,1.78,3.04
3,Albania,3.61,3.44,2.04,1.93,1.61,1.87,1.29,1.98,2.03,1.41,1.62,2.04,6.73
4,Andorra,14.48,13.48,10.28,8.78,7.3,9.16,30.69,29.84,19.63,17.08,22.28,25.77,21.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Yemen,11.17,19.54,9.89,10.97,8.16,22.0,21.3,30.4,33.65,15.43,19.59,26.02,29.07
206,South Africa,4.21,4.99,5.65,5.74,6.1,4.59,6.33,5.28,4.61,4.12,3.28,4.56,6.87
207,Zambia,8.5,8.66,6.58,6.98,7.81,10.11,17.87,6.58,7.49,9.15,15.73,22.02,10.99
208,Zimbabwe,3.04,3.47,3.72,1.63,-0.21,-2.41,-1.56,0.91,10.61,255.29,557.21,98.55,193.4


**Transformation 4: Overall Change**
<br>
<br>
Something that can be measured overall is the total change in CPI over the 2010-2022 timespan. Since this dataframe shows each year's total increase of CPI, the sum of all values for a given country is then the total change in CPI over the timespan. Another column can be added to show that information.

In [28]:
# Getting the sums of each row
sums = []
for i in range(210):
    sum = UN.iloc[i][1:].sum()
    sums.append(sum)

UN['Total Change'] = sums
UN

,Country,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,Total Change
0,Aruba,2.08,4.38,0.57,-2.37,0.42,0.47,-0.93,-1.03,3.63,3.94,-1.31,0.73,5.59,16.17
1,Afghanistan,NaN,12.17,-28.64,-9.9,4.67,-1.55,2.12,-10.57,0.63,2.3,5.6,5.13,13.71,-4.33
2,Angola,2.15,4.7,1.43,0.19,-0.28,-0.94,-0.57,1.34,0.38,0.75,-0.47,1.78,3.04,13.50
3,Albania,3.61,3.44,2.04,1.93,1.61,1.87,1.29,1.98,2.03,1.41,1.62,2.04,6.73,31.60
4,Andorra,14.48,13.48,10.28,8.78,7.3,9.16,30.69,29.84,19.63,17.08,22.28,25.77,21.36,230.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Yemen,11.17,19.54,9.89,10.97,8.16,22.0,21.3,30.4,33.65,15.43,19.59,26.02,29.07,257.19
206,South Africa,4.21,4.99,5.65,5.74,6.1,4.59,6.33,5.28,4.61,4.12,3.28,4.56,6.87,66.33
207,Zambia,8.5,8.66,6.58,6.98,7.81,10.11,17.87,6.58,7.49,9.15,15.73,22.02,10.99,138.47
208,Zimbabwe,3.04,3.47,3.72,1.63,-0.21,-2.41,-1.56,0.91,10.61,255.29,557.21,98.55,193.4,1123.65


**Transformation 5: Column Re-Naming**
<br>
<br>
The final goal of this Term Project is to merge the datasets together and make a database. Since CSV' data has columns of the form YYYYQ#, we will want to make sure that te data from this dataframe shows that the data came from a different source. So I will add UN to each of the column names with numerical data.

In [30]:
# New Column Names
names = ['Country', '2010UN', '2011UN', '2012UN', '2013UN', '2014UN', '2015UN',
         '2016UN', '2017UN', '2018UN', '2019UN', '2020UN', '2021UN', '2022UN', 'Total Change UN']

# Renaming the columns
UN.columns = names
UN

,Country,2010UN,2011UN,2012UN,2013UN,2014UN,2015UN,2016UN,2017UN,2018UN,2019UN,2020UN,2021UN,2022UN,Total Change UN
0,Aruba,2.08,4.38,0.57,-2.37,0.42,0.47,-0.93,-1.03,3.63,3.94,-1.31,0.73,5.59,16.17
1,Afghanistan,NaN,12.17,-28.64,-9.9,4.67,-1.55,2.12,-10.57,0.63,2.3,5.6,5.13,13.71,-4.33
2,Angola,2.15,4.7,1.43,0.19,-0.28,-0.94,-0.57,1.34,0.38,0.75,-0.47,1.78,3.04,13.50
3,Albania,3.61,3.44,2.04,1.93,1.61,1.87,1.29,1.98,2.03,1.41,1.62,2.04,6.73,31.60
4,Andorra,14.48,13.48,10.28,8.78,7.3,9.16,30.69,29.84,19.63,17.08,22.28,25.77,21.36,230.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,Yemen,11.17,19.54,9.89,10.97,8.16,22.0,21.3,30.4,33.65,15.43,19.59,26.02,29.07,257.19
206,South Africa,4.21,4.99,5.65,5.74,6.1,4.59,6.33,5.28,4.61,4.12,3.28,4.56,6.87,66.33
207,Zambia,8.5,8.66,6.58,6.98,7.81,10.11,17.87,6.58,7.49,9.15,15.73,22.02,10.99,138.47
208,Zimbabwe,3.04,3.47,3.72,1.63,-0.21,-2.41,-1.56,0.91,10.61,255.29,557.21,98.55,193.4,1123.65


**Implications from the Data Wrangling**
<br>
<br>
When working with this data, it was important to make sure that a static copy of the data was saved since Wikipedia is editable by the site's nature. While edits made by lay-users are subject to approval, changes to the page itself could end up breaking parts of this code since they are specific to using a set table with set information.
<br>
<br>
Ethically, there should be no concerns with respect to using this data since it does not implicate or endanger any specific individuals. However, it should be noted that the communication of the data should be done conscientiously. The data from the CSV used for this term project used the World Bank's total value of the Consumer Price Index each year while this data comes from the UN's change in the value of the Consumer Price Index (but not in terms of percents). So the values measure the same phenomena across the 2010-2022 timespan on an anual basis but are presented differently. Further analysis of the data and the communication of the results should take care in how data is presented and used.

In [32]:
# Printing the dataframe to CSV
UN.to_csv('UN.csv', index=False)